# Image Recognition Project - Structural Defect Recognition
---------------------------------------------------------------
## Data Collection

### Section Objectives
 - Find relevant dataset from Kaggle
 - Collect the data
 - Preprocess data, checking for outlier images or irrelevant files
 - Perform a manual data check
 - Divide dataset into the following subsets: Train, Test and Validation; at the ratio 0.7, 0.2, 0.1
 

---------------------------------------------------------------

### Importing Packages

In [ ]:
%pip install -r /workspaces/ML_Project_Image_Recognition/requirements.txt

In [ ]:
import numpy
import os

### Setting Working Directory

In [ ]:
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir('workspaces/ML_Project_Image_Recognition')
print("This is your set Working Directory")

### Installing Kaggle


In [ ]:
%pip install kaggle==upgrade kaggle